## [1] 유튜브 인기동영상 데이터 ##

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
#['title', 'channelTitle', 'categoryId', 'view_count', #'likes', 'dislikes', 'comment_count', 'channelId', 'trending_date2']

In [ ]:
# Q1) 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)
df.value_counts('channelTitle').reset_index().sort_values(0, ascending = False)[channelTitle].head(10).tolist()

In [ ]:
# Q2) 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라
df[df['dislikes'] > df['likes']]['channelTitle'].unique().tolist()

In [ ]:
## 오답 : value_counts
# Q3) 채널명을 바꾼 케이스가 있는지 확인하고 싶다.
#     channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라
(df[['channelTitle','channelId']].drop_duplicates()['channelId'].value_counts() > 1).sum()

In [ ]:
## 오답 : 날짜형 데이터 
# Q4) 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?
df[pd.to_datetime(df['trending_date2']).dt.day_name() == 'Sunday']['categoryId'].value_counts().index[0]

In [ ]:
## 오답 : 피벗 테이블 생성
# Q5) 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라
df['trending_date2'] = pd.to_datetime(df['trending_date2'])  # 날짜(str) 컬럼 타입을 datetime 으로 변경

group = df.groupby([df['trending_date2'].dt.day_name(),'categoryId'],as_index=False).size()
group.pivot(index='categoryId',columns='trending_date2')  # 요일별(column) 카테고리 id(index)

In [ ]:
## 오답 : 비율 계산 (~~ 대비)
# Q6) 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.
#     viewcount대비 댓글수가 가장 높은 영상을 확인하라(view_count값이 0인 경우는 제외한다)
df2 = df[df['view_count'] != 0][['title','comment_count','view_count']]
df2["reaction"] = (df['comment_count']/df['view_count'])
str(df2.sort_values(by = ['reaction'], ascending = False).iloc[1].title)

In [ ]:
# Q7) 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.
#     viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)
df2 = df[df['view_count'] != 0][['title','comment_count','view_count']]
df2["reaction"] = (df['comment_count']/df['view_count'])
str(df2[df2["reaction"] != 0].sort_values(by = ['reaction']).iloc[1].title)

In [ ]:
# Q8) like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)
df2 = df[(df['dislikes'] != 0) & (df['likes'] != 0)][['title','dislikes','likes']]
df2['ratio'] = df['dislikes']/df['likes']
str(df2.sort_values(by = ['ratio']).iloc[0,0])

In [ ]:
# Q9) 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)10
df.groupby(['channelTitle', 'channelId']).size().sort_values(by = ['size'], ascending = False)['channelTitle'].iloc[0]

In [ ]:
## 오답 : sum함수
# Q10) 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?
(df[['title','channelId']].value_counts()>=20).sum()

## [2] 유튜브 공범컨텐츠 동영상 데이터

In [ ]:
import pandas as pd
import datetime
channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
# channel : 'channelid', 'subcnt', 'viewcnt', 'videocnt', 'ct', 'channelname'
# video : 'videopk', 'viewcnt', 'likecnt', 'dislikecnt', 'favoritecnt', 'cmcnt', 'ct', 'videoname'   

In [ ]:
# Q1) 각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고,
channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])

# video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라
video['videoname'].value_counts()

In [ ]:
## 오답 : 중복제거
# Q2) 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라
video.sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last')[['viewcnt','videoname','ct']]

In [ ]:
## 오답 : 날짜형 비교
# Q3) Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라
channel[channel['ct'] >= pd.to_datetime('2021-10-03')].sort_values(['ct']).drop_duplicates('channelname')

In [ ]:
## 오답 : 날짜형 비교, 데이터프레임 병합
# Q4) 각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라
subs_start = channel[channel['ct'].dt.strftime('%Y-%m-%d %H') =='2021-10-03 03'][['channelname', 'subcnt']]
subs_end = channel[channel['ct'].dt.strftime('%Y-%m-%d %H') =='2021-11-01 15'][['channelname', 'subcnt']]

subs_start.rename(columns = {'subcnt' : 'subcnt_start'})
subs_end.rename(columns = {'subcnt' : 'subcnt_end'})

chann_sub = pd.merge(subs_start, subs_end)
chann_sub['del'] = (chann_sub['subcnt_end'] - chann_sub['subcnt_start'])
chann_sub[['channelname', 'del']]

In [ ]:
## 오답 : 문자형 컬럼에서 데이터 추출 및 날짜 계산
# Q5) 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요수, 댓글수가 수집된것으로 알려졌다.
#     공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력하라
df = video[video['videoname'].str.contains('1')].sort_values('ct').reset_index(drop = True)
df[(df['ct'].diff(1) >= datetime.timedelta(minutes=20)) | (df['ct'].diff(1) <= datetime.timedelta(minutes=5))]

In [ ]:
# Q6) 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라
df = video.sort_values(by = ['videoname', 'ct']).drop_duplicates('videoname')[['videoname', 'ct']].reset_index(drop = True)
df['date'] = df['ct'].dt.strftime("%Y-%m-%d")
df[['videoname','date']]

In [ ]:
# Q7) “공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다.
#      공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라
v_df = video[video['ct'].dt.hour == 21].sort_values(by = ['videoname', 'ct']).drop_duplicates('videoname')
v_df.sort_values('viewcnt', ascending = False).reset_index(drop = True)[['viewcnt', 'ct', 'videoname']]

In [ ]:
# Q8) video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 
#     videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라
df = video.sort_values('ct', ascending = False).drop_duplicates('videoname')
target = video.sort_values('ct').drop_duplicates('videoname',keep='last')
df['ratio'] = df['dislikecnt']/df['likecnt']
df[['videoname', 'ratio']].sort_values('ratio').reset_index(drop = True)

In [ ]:
## 오답 : 데이트 타임 변경, groupby 함수
# Q9) 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오
start = pd.to_datetime("2021-11-01 00:00:00")
end = pd.to_datetime("2021-11-01 15:00:00")

df = video.loc[(video["ct"] >= start) & (video['ct'] <= end)].reset_index(drop=True)[['videoname', 'viewcnt']]
df = df.groupby('videoname').agg([min, max])
df.columns = ['min', 'max']
df['증가율'] = df['max'] - df['min']
df = df.drop(columns = ['min', 'max'])

## [3] 월드컵 출전선수 골기록 데이터

In [ ]:
import pandas as pd
# 데이터 로드
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
# ['Player', 'Goals', 'Years', 'Country']

In [ ]:
# Q1) 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수를 데이터프레임형태로 출력하라
df.groupby('Country').sum().sort_values('Goals',ascending=False).head(5)

In [ ]:
# Q2) 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라
df.groupby('Country').size().sort_values(ascending=False).head(5)

In [ ]:
## 오답 :  문자형 컬럼 split & length
# Q3) 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라

# df2['yearLst'] = ['2002', '2006', '2010', '2014'], [''2006',  '2014']
df['yearLst'] = df.Years.str.split('-')

# 1안
count = 0
for i in range(len(df)):
    if len(df.yearLst[i]) == 4:
        count = count + 1

# 2안
df['LenCup'] = df['yearLst'].str.len()
df['LenCup'].value_counts()[4]

In [ ]:
# Q4) Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라
len(df2[(df2.LenCup==2) & (df2.Country =='Yugoslavia')])

In [ ]:
## 오답 : 문자형 컬럼 특정값을 포함하는 경우
# Q5) 2002년도에 출전한 전체 선수는 몇명인가?
len(df[df['Years'].str.contains('2002')])

In [ ]:
## 오답 : 문자형 컬럼 특정값을 포함하는 경우 .str.contains
# Q6) 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)
df[df['Player'].str.lower().str.contains('carlos')]

In [ ]:
# Q7) 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?
df[df['LenCup'] == 1].sort_values(by = 'Goals', ascending = False).iloc[0]['Player']

In [ ]:
# Q8) 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?
df[df['LenCup'] == 1].Country.value_counts().index[0]

## [4] 서울시 따릉이 이용정보 데이터

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
# '대여일자', '대여시간', '대여소번호', '대여구분코드', '성별', '연령대코드', '이용건수', '운동량',
# '탄소량', '이동거리', '사용시간'

In [ ]:
# Q1) 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라.
df['대여일자'].value_counts().to_frame().sort_values(by = '대여일자', ascending = False).index[0]

In [ ]:
# Q2) 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’)
#   ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()

df['day_name'].value_counts().to_frame()

In [ ]:
# Q3) 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라
df = df.groupby(['day_name','대여소번호']).size().to_frame('size').sort_values('size', ascending = False).reset_index()
df.drop_duplicates('day_name')

In [ ]:
## 오답 : 문자열 컬럼 특정 값을 포함하는 경우  isin
# Q4) 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라.
#     일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

daily = df[df.대여구분코드.isin(['일일권','일일권(비회원)'])].연령대코드.value_counts().sort_index()
total = df.연령대코드.value_counts().sort_index()
ratio = (daily /total).sort_values(ascending=False)
ratio

In [ ]:
# Q5) 연령대별 평균 이동거리를 구하여라
df[['연령대코드', '이동거리']].groupby('연령대코드').mean()

In [ ]:
# Q6) 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.
#     최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후
#     1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라
data = df[df['연령대코드'] == '20대'].reset_index(drop=True)
탄소량 = data[data['이동거리'] >= data['이동거리'].mean()].sort_values(by = ['대여일자', '대여소번호'], ascending = False)
탄소량 = 탄소량.reset_index(drop = True).iloc[1:201]['탄소량'].tolist()
탄소량 = [float(i) for i in 탄소량]

round(sum(탄소량)/len(탄소량), 3)

In [ ]:
# Q7) 6월 7일 ~10대의 “이용건수”의 중앙값은?
data = df[(df['대여일자'] ==  '2021-06-07') & (df['연령대코드'] == '~10대')].reset_index(drop=True)
data['이용건수'].median()

In [ ]:
# Q8) 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후
#     각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라
df[(df['대여일자'].dt.day_name().isin(['Monday', 'Tuseday', 'Wednesday', 'Tuesday', 'Friday'])) & (df['대여시간'].isin([6,7,8]))]
data = data[['대여소번호', '이용건수', '대여시간']].reset_index(drop = True).sort_values(['대여시간','이용건수'],ascending=False)
data.groupby(['대여시간']).head(3)

In [ ]:
# Q9) 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라
df[df['이동거리'] >= df['이동거리'].mean()]['이동거리'].std()

In [ ]:
# Q10) 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라
df['성별'] = df['성별'].str.lower() 

df[['성별', '이동거리']].groupby('성별').mean()

## [5] 전세계 행복도 지표 데이터

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')

In [ ]:
# Q1) 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다.
#    각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라
df[df.행복랭킹 ==10]['점수'].mean()

In [ ]:
# Q2) 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라
data = df[df['행복랭킹'] <= 50][['년도', '점수']]
data.groupby('년도').mean()

In [ ]:
# Q3) 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라
result = len(df[['행복랭킹','나라명']]) - len(df[['행복랭킹','나라명']].drop_duplicates())

In [ ]:
## 오답 : 상관계수 corr()
# Q4) 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고,
#    내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시하라
data = df[df['년도'] == 2019].corr().unstack().to_frame('crr').reset_index().rename({'level_0' : 'v1', 'level_1':'v2'})
data[data['crr'] != 1].drop_duplicates('crr').sort_values('crr', ascending = False).reset_index(drop = True).head(5)

In [ ]:
# Q5) 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라
df.groupby('년도').tail(5).groupby('년도').mean()[['점수']]

In [ ]:
# Q6) 2019년 데이터를 추출하고 해당데이터의
#    상대 GDP 평균 이상의 나라들과 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라
upper = df[df['상대GDP'] >= df['상대GDP'].mean()]['점수'].mean()
lower = df[df['상대GDP'] <= df['상대GDP'].mean()]['점수'].mean()
abs(upper - lower)

In [ ]:
# Q7) 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라
data = df.sort_values(by = ['년도','부패에 대한인식'], ascending = False)
data.groupby('년도').head(20).groupby('년도').mean()[['부패에 대한인식']]

In [ ]:
# Q8) 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라
# 1안)
contry_18 = df[df['행복랭킹'] <= 50]['나라명'].tolist()
contry_19 = df[df['나라명'].isin(contry_18)]
contry_19[contry_19['행복랭킹'] > 50]

# 2안)
set(df[(df.년도==2018) & (df.행복랭킹 <=50)].나라명) - set(df[(df.년도==2019) & (df.행복랭킹 <=50)].나라명)

In [ ]:
# Q9) 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치는?
score_18= df[df['년도'] == 2018][['점수', '나라명']].rename(columns = {'점수' : '점수_2018'}).reset_index(drop = True)
score_19= df[df['년도'] == 2019][['점수', '나라명']].rename(columns = {'점수' : '점수_2019'}).reset_index(drop = True)

data = pd.merge(score_18, score_19, how='inner',on='나라명').dropna()
data['점수차'] = data['점수_2019'] - data['점수_2018'] 
data.sort_values(by = '점수차' , ascending = False).head(1)[['나라명', '점수차']].head(1)

## [6] 지역구 에너지 소비량 데이터

In [ ]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')

In [ ]:
# Q1) DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라
df['month'] = pd.to_datetime(df['DateTime']).dt.month
df.groupby('month').size()

In [ ]:
# Q2) 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라
data = df[df['month'] == 3]
data['hour'] = pd.to_datetime(data['DateTime']).dt.hour
data.groupby('hour')['Temperature'].mean().sort_values().iloc[0]

In [ ]:
# Q3) 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라
data.groupby('hour')['Temperature'].mean().sort_values(ascending = False).iloc[0]

In [ ]:
# Q4) Zone 1 Power Consumption 컬럼의 value값의 크기가
#     Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균을 구하여라
df[df['Zone 1 Power Consumption'] > df['Zone 2  Power Consumption']]['Humidity'].mean()

In [ ]:
# Q5) 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라
data = df[['Zone 1 Power Consumption', 'Zone 2  Power Consumption', 'Zone 3  Power Consumption']]
data.corr().unstack()

In [ ]:
# Q6) Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C,
#     그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라
df['GRD'] = ['' for i in range(len(df))]
for i in range(len(df)):
    if df['Temperature'].iloc[i] < 10:
        df['GRD'].iloc[i] = 'A'
    elif (df['Temperature'].iloc[i] >= 10) & (df['Temperature'].iloc[i] < 20):
        df['GRD'].iloc[i] = 'B'
    elif (df['Temperature'].iloc[i] >= 20) & (df['Temperature'].iloc[i] < 30):
        df['GRD'].iloc[i] = 'C'
    else:
        df['GRD'].iloc[i] = 'D'

df.value_counts('GRD')

In [ ]:
# Q7) 6월 데이터중 12시의 Temperature의 표준편차를 구하여라
data = df[pd.to_datetime(df['DateTime']).dt.month == 6]
data[pd.to_datetime(data['DateTime']).dt.hour == 12]['Temperature'].std()

In [ ]:
# Q8) 6월 데이터중 12시의 Temperature의 분산을 구하여라
data[pd.to_datetime(data['DateTime']).dt.hour == 12]['Temperature'].var()

In [ ]:
# Q9) Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?
df[df['Temperature'] >= df['Temperature'].mean()].sort_values('Temperature').iloc[3]['Humidity']

In [ ]:
# Q10) Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?
df[df['Temperature'] >= df['Temperature'].median()].sort_values('Temperature').iloc[3]['Humidity']

## [7] 포켓몬 정보 데이터

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')

In [ ]:
# 1) Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라
abs(df[df['Legendary'] == True]['HP'].mean() - df[df['Legendary'] == False]['HP'].mean())

In [ ]:
# 2) Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?
df.value_counts('Type 2').index[0]

In [ ]:
# 3) 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?
df[df['Type 1'] == df.value_counts('Type 1').index[0]]['Attack'].mean()\
     /df[df['Type 1'] == df.value_counts('Type 1').index[0]]['Defense'].mean()

In [ ]:
# 4) 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?
df[df['Legendary'] == True][['Generation', 'Legendary']].groupby('Generation').size().sort_values(ascending = False).index[0]

In [ ]:
# 5) ‘HP’, 'Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’
#     간의 상관 계수중 가장 절댓값이 큰 두 변수와 그 값을 구하여라
data = df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]
data_crr = data.corr().unstack().to_frame('crr').reset_index()
data_crr = data_crr[data_crr['crr'] != 1].drop_duplicates('crr')
data_crr['crr'] = abs(data_crr['crr'])
data_crr.sort_values('crr', ascending = False).reset_index(drop = True).iloc[0]

In [ ]:
# 6) 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균을 구하여라
data = df.sort_values(['Generation','Attack'])
data.groupby('Generation').head(3).Attack.mean()

In [ ]:
# 7) 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라
data = df.sort_values(['Generation','Attack'], ascending = False)
data.groupby('Generation').head(5).Attack.mean()

In [ ]:
## 오답
# 8) 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?
df[['Type 1','Type 2']].value_counts().head(1)

In [ ]:
# 9) 한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?
data = df[['Type 1','Type 2']].value_counts().to_frame('count')
len(data[data['count'] == 1])

In [ ]:
# 10) 한번씩만 존재하는 (Type1 , Type2)의 쌍을 각 세대(Generation)은 각각 몇개씩 가지고 있는가?
data = df[['Type 1','Type 2']].value_counts().to_frame('count')
data = data[data['count'] == 1].reset_index()

type_1 = data['Type 1'].tolist()
type_2 = data['Type 2'].tolist()

result = pd.DataFrame()
for i in range(len(type_1)):
    data_t = df[(df['Type 1'] == type_1[i]) & (df['Type 2'] == type_2[i])]
    result = pd.concat([result, data_t])

result.groupby('Generation').size()

## [8] 대한민국 체력장 데이터

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv')

In [ ]:
# 1) 전체데이터의 수축기혈압(최고) - 이완기혈압(최저)의 평균을 구하여라
(df['수축기혈압(최고) : mmHg'] - df['이완기혈압(최저) : mmHg']).mean()

In [ ]:
# 2) 50~59세의 신장평균을 구하여라
df[(df['측정나이'] >= 50) & (df['측정나이'] <= 59)]['신장 : cm'].mean()

In [ ]:
# 3) 연령대 (20~29 : 20대 …) 별 인원수를 구하여라
df['연령대'] = (df['측정나이']//10)*10
df.groupby('연령대').size()

In [ ]:
# 4) 연령대 (20~29 : 20대 …) 별 등급의 숫자를 데이터 프레임으로 표현하라
df.groupby(['연령대', '등급']).size().to_frame('count').reset_index()

In [ ]:
# 5) 남성 중 A등급과 D등급의 체지방률 평균의 차이(큰 값에서 작은 값의 차)를 구하여라
abs(df[(df['측정회원성별'] == 'M') & (df['등급'] == 'A')]['체지방율 : %'].mean()
              - df[(df['측정회원성별'] == 'M') & (df['등급'] == 'D')]['체지방율 : %'].mean())

In [ ]:
# 6) 여성 중 A등급과 D등급의 체중의 평균의 차이(큰 값에서 작은 값의 차)를 구하여라
abs(df[(df['측정회원성별'] == 'F') & (df['등급'] == 'A')]['체중 : kg'].mean() 
               - df[(df['측정회원성별'] == 'F') & (df['등급'] == 'D')]['체중 : kg'].mean())

In [ ]:
# 7) bmi는 자신의 몸무게(kg)를 키의 제곱(m)으로 나눈값이다. 데이터의 bmi 를 구한 새로운 컬럼을 만들고 남성의 bmi 평균을 구하여라
df['BMI'] = df['체중 : kg']/(df['신장 : cm']/100) **2
df[df['측정회원성별'] == 'M']['BMI'].mean()

In [ ]:
# 8) bmi보다 체지방율이 높은 사람들의 체중평균을 구하여라
df[df['체지방율 : %'] > df['BMI']]['체중 : kg'].mean()

In [ ]:
# 9) 남성과 여성의 악력 평균의 차이를 구하여라
abs(df[(df['측정회원성별'] == 'M')]['악력D : kg'].mean() - df[(df['측정회원성별'] == 'F')]['악력D : kg'].mean())

In [ ]:
# 10) 남성과 여성의 교차윗몸일으키기 횟수의 평균의 차이를 구하여라
abs(df[(df['측정회원성별'] == 'M')]['교차윗몸일으키기 : 회'].mean()
                - df[(df['측정회원성별'] == 'F')]['교차윗몸일으키기 : 회'].mean())

## [9] 기온 강수량 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")

In [ ]:
# 1) 여름철(6월,7월,8월) 이화동이 수영동보다 높은 기온을 가진 시간대는 몇개인가
# df['time'] = pd.to_datetime(df['time']).dt.date
data = df[pd.to_datetime(df['time']).dt.month.isin([6, 7, 8])]
len(data[data['이화동기온'] > data['수영동기온']])

In [ ]:
# 2) 이화동과 수영동의 최대강수량의 시간대를 각각 구하여라
df[df['이화동강수'] == df['이화동강수'].max()]['time'].iloc[0]
df[df['수영동강수'] == df['이화동강수'].max()]['time'].iloc[0]

## [10] 서비스 이탈 예측 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv")

In [ ]:
# 1) 남성 이탈(Exited)이 가장 많은 국가(Geography)는 어디이고 이탈 인원은 몇명인가s
df[df['Gender'] == 'Male'].groupby('Geography').Exited.sum().to_frame('count').sort_values('count', ascending = False).iloc[0]

In [ ]:
# 2) 카드를 소유(HasCrCard ==1)하고 있으면서 활성멤버(IsActiveMember ==1) 인 고객들의 평균 나이를 소숫점이하 4자리까지 구하여라?
round(df[(df['HasCrCard'] == 1) & (df['IsActiveMember'] == 1)]['Age'].mean(), 4)

In [ ]:
# 3) Balance 값이 중간값 이상을 가지는 고객들의 CreditScore의 표준편차를 소숫점이하 3자리까지 구하여라
round(df[df['Balance'] >= df['Balance'].median()]['CreditScore'].std() , 3)

## [11] 성인 건강검진 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")

In [ ]:
# 1) 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼(‘혈압차’) 으로 생성하고,
#    연령대 코드별 각 그룹 중 ‘혈압차’ 의 분산이 5번째로 큰 연령대 코드를 구하여라
df['혈압차'] = df['수축기혈압'] - df['이완기혈압']
df[['연령대코드(5세단위)', '혈압차']].groupby('연령대코드(5세단위)').var().sort_values('혈압차', ascending = False).index[4]

In [ ]:
# 2) 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다.
#    일반적으로 0.58이상이면 비만으로 분류한다. 데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라
df['WHtR'] = df['허리둘레']/df['신장(5Cm단위)']

all = len(df[df['WHtR'] >= 0.58])
male = len(df[(df['WHtR'] >= 0.58) & (df['성별코드'] == 'M')])
fmale = len(df[(df['WHtR'] >= 0.58) & (df['성별코드'] == 'F')])
male/fmale

## [12] 자동차 보험가입 예측데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")

In [ ]:
# 1) Vehicle_Age 값이 2년 이상인 사람들만 필터링 하고 그중에서
#    Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들을 찾고, 그들의 Vintage값의 평균을 구하여라
df[(df['Vehicle_Age'] == '> 2 Years') & (df['Annual_Premium'] >=  df['Annual_Premium'].median())]['Vintage'].mean()

In [ ]:
# 2) vehicle_age에 따른 각 성별(gender)그룹의 Annual_Premium값의 평균을 구하여 아래 테이블과 동일하게 구현하라
df.groupby(['Vehicle_Age', 'Gender'])['Annual_Premium'].mean().unstack()

## [13] 핸드폰 가격 예측 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv")

In [ ]:
## 오답 : groupby
# 1) price_range 의 각 value를 그룹핑하여 각 그룹의 n_cores 의 빈도가 가장높은 value와 그 빈도수를 구하여라]
df[['price_range', 'n_cores']].groupby(['price_range', 'n_cores']).size().sort_values(0).groupby(level=0).tail(1)

In [ ]:
# 2) price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수를 구하여라
c_data = df[df['price_range'] == 3].corr().unstack().to_frame('corr').dropna().reset_index()
c_data[c_data['corr'] != 1].sort_values('corr', ascending = False).drop_duplicates().iloc[1]

## [14] 비행탑승 경험 만족도 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv")

In [ ]:
## 오답 : pivot 테이블
# 1) Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중
#   neutral or dissatisfied’ 보다 ‘satisfied’의 수가 더 높은 Class는 어디 인가?
data = df[df['Arrival Delay in Minutes'].isnull()][['Class', 'satisfaction']].value_counts().to_frame('count').reset_index()
dtat_2 = data.pivot(index='Class',columns='satisfaction')
dtat_2[dtat_2['count']['neutral or dissatisfied'] < dtat_2['count']['satisfied']].index

## [15] 수질 음용성 여부 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv")

In [ ]:
# 1) ph값은 상당히 많은 결측치를 포함한다. 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값은?
data = df[df['ph'].isnull() == False]
data[data['ph'] <= data.quantile(0.25)['ph']]['ph'].mean()

## [16] 의료비용 예측 데이터

In [ ]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")

In [ ]:
# 1) 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?
no = train[train['smoker'] == 'no']['charges'].quantile(0.9)
yes = train[train['smoker'] == 'yes']['charges'].quantile(0.9)

abs(train[(train['smoker'] == 'no') & (train['charges'] >= no)]['charges'].mean() - 
train[(train['smoker'] == 'yes') & (train['charges'] >= yes)]['charges'].mean())

## [17] 킹카운티 주거지 가격예측문제 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice//train.csv")

In [ ]:
# 1) bedrooms 의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10%값의 차이를 구하여라
df[df['bedrooms'] == df['bedrooms'].value_counts().index[0]]['price'].quantile(0.9) 
            - df[df['bedrooms'] == df['bedrooms'].value_counts().index[0]]['price'].quantile(0.1)

## [18] 대학원 입학 가능성 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")

In [ ]:
## 오답 : 랜덤포레슨트
# 1) Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때,
#    랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)
from sklearn.ensemble import RandomForestRegressor

df_t = df.drop('Serial No.',axis=1)
x = df_t.drop('Chance of Admit',axis=1) ## 독립변수
y = df_t['Chance of Admit']  # 종속변수

ml = RandomForestRegressor()
ml.fit(x,y)

result = pd.DataFrame({'importance':ml.feature_importances_},x.columns).sort_values('importance',ascending=False)

## [19] 레드와인 퀄리티 예측 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv")

In [ ]:
# 1) quality 값이 3인 그룹과 8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이를 구할때 그값이 가장 큰 컬럼명을 구하여라
(df.loc[df.quality ==8].std() -df.loc[df.quality ==3].std()).sort_values(ascending = False).index[0]

## [20] 약물 분류 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv")

In [ ]:
## 방법 2번 참고 필요
# 1) 남성들의 연령대별 (10살씩 구분 0~9세 10~19세 …) Na_to_K값의 평균값을 구해서 데이터 프레임으로 표현하여라
data = df[df['Sex'] == 'M']

# 방법1)
data['연령대'] = [''] *len(data)
for i in range(len(data)):
    if data['Age'].iloc[i] < 9:
        data['연령대'].iloc[i] = '0대'
    elif (data['Age'].iloc[i] >= 10) & (data['Age'].iloc[i] < 19):
        data['연령대'].iloc[i] = '10대'
    elif (data['Age'].iloc[i] >= 20) & (data['Age'].iloc[i] < 30):
        data['연령대'].iloc[i] = '20대'
    elif (data['Age'].iloc[i] >= 30) & (data['Age'].iloc[i] < 40):
        data['연령대'].iloc[i] = '30대'
    elif (data['Age'].iloc[i] >= 40) & (data['Age'].iloc[i] < 50):
        data['연령대'].iloc[i] = '40대'
    elif (data['Age'].iloc[i] >= 50) & (data['Age'].iloc[i] < 60):
        data['연령대'].iloc[i] = '50대'
    elif (data['Age'].iloc[i] >= 60) & (data['Age'].iloc[i] < 70):
        data['연령대'].iloc[i] = '60대'   
    elif (data['Age'].iloc[i] >= 70) & (data['Age'].iloc[i] < 80):
        data['연령대'].iloc[i] = '70대'  

# 방법2)
data['연령대'] = data['Age']//10 *10
        
data.groupby('연령대')['Na_to_K'].mean().to_frame()

## [21] 사기회사 분류 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")

In [ ]:
# 1) 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라
df.groupby('Risk')[['Score_A', 'Score_B']].mean()

## [22] 센서데이터 동작유형 분류 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv")

In [ ]:
# 1) pose값에 따른 각 motion컬럼의 중간값의 가장 큰 차이를 보이는 motion컬럼은 어디이며 그값은?
data = df.groupby('pose').median().T
data.columns = ['pose_0', 'pose_1']

data['diff'] =  abs(data['pose_0'] -  data['pose_1'] )
data[data['diff'] == data['diff'].max()]

## [23] 현대 차량 가격 분류문제 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv")

In [ ]:
# 1) 정보(row수)가 가장 많은 상위 3차종의 price값의 각 평균값은?
data = df[df['model'].isin(df.groupby('model').size().sort_values(ascending = False).index[0:3].tolist())]
data[['model', 'price']].groupby('model').mean()

## [24] 당뇨여부 판단데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv")

In [ ]:
# 1) Outcome 값에 따른 각 그룹의 각 컬럼의 평균 차이를 구하여라
data = df.groupby('Outcome').mean().T
data['diff'] = abs(data[0] - data[1])

## [25] 넷플릭스 주식 데이터

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv")

In [ ]:
# 1) 매년 5월달의 open가격의 평균값을 데이터 프레임으로 표현하라
df['Date'] = pd.to_datetime(df['Date'])
data =df[df['Date'].dt.month == 5]
data['year'] = data['Date'].dt.year
data.groupby('year')['Open'].mean().to_frame()

## [26] NBA 선수 능력치 데이터

In [ ]:
import pandas as pd
pd.set_option('display.max_columns',50)
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv",encoding='latin',sep=';')

In [ ]:
# 1) Tm 컬럼은 각 팀의 이름을 의미한다. TOR팀의 평균나이를 소수 4째 자리까지 구하여라
round(df[df['Tm'] == 'TOR']['Age'].mean(), 4)

In [ ]:
## 오답 : 문자형 컬럼 
# 2) 선수들의 이름은 first_name+ 공백 + last_name으로 이루어져 있다. 가장 많은 first_name은 무엇이며 몇 회 발생하는지 확인하라
df['Player'].str.split().str[0].str.lower().value_counts().head(1)

In [ ]:
# 3) PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라
df.groupby('Pos')['PTS'].mean()

In [ ]:
## 오답 : groupby
# 4) G컬럼은 참여한 경기의 숫자이다. 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균을 구하여라
df.sort_values(['Tm','G']).groupby('Tm').tail(1)['G'].mean()

In [ ]:
# 5) Tm의 값이 MIA이며 Pos는 C또는 PF인 선수의 MP값의 평균은?
df[(df['Tm'] == 'MIA') & (df['Pos'].isin(['C', 'PF']))]['MP'].mean()

In [ ]:
# 6) 전체 데이터중 G의 평균값의 1.5배 이상인 데이터들만 추출했을때 3P값의 평균은?
df[df['G'] >= df['G'].mean()*1.5]['3P'].mean()

In [ ]:
# 7) Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이는?
upper = df[df['Age'] >= df['Age'].mean()]['G'].mean()
lower = df[df['Age'] < df['Age'].mean()]['G'].mean()
upper - lower

In [ ]:
# 8) 평균나이가 가장 젊은 팀은 어디인가
df.groupby('Tm')['Age'].mean().sort_values().index[0]